<h1>Full-length 16S Metagenomics Pipeline</h1>

## Starting Files 

1. This Jupyter notebook
2. Directories for organizing the data. To make the folders, run the following code block:

In [2]:
!mkdir \
data \
results \
results/1-data-prep \
results/2-asv-infer \
results/3-tax-assign \
results/4-rarefy \
results/5-diversity

mkdir: cannot create directory ‘data’: File exists


# <font color = 'gray'>Step 1: Data Preparation</font>

### Importing the sequencing reads into the pipeline

To begin this pipeline, ensure that the sequencing reads have been transferred into the <font face="Consolas">**data folder**</font>

### Making a manifest file

Before we import our data, we have to make a **manifest file** that contains links to the PacBio Circular Consensus Sequencing (CCS) reads of each sample.

In [3]:
import pandas as pd
import glob
import os

sampleIDs, abs_path = [],[]
fpath= os.getcwd()+"/data/"
for filepath in (glob.glob(fpath+"*.fastq")):
    sample = filepath.split("/")[-1].rsplit("-", 2)[0]

    if sample not in sampleIDs:
        sampleIDs.append(sample)
    if filepath not in abs_path:
        abs_path.append(filepath)

manifest =  pd.DataFrame({'sampleID': sorted(sampleIDs), 'absolute-filepath': sorted(abs_path)})
manifest = manifest.sort_index(axis=1, ascending=False)
with open('results/1-data-prep/manifest.txt', 'w') as m:
    print(manifest.to_csv(sep='\t', index=False, header=True), file=m)

### Converting the sequencing reads into QIIME2 artifacts

To begin this pipeline, ensure that the sequencing reads have been transferred into the <font face="Consolas">**data folder**</font>

In [6]:
!qiime tools import \
    --type SampleData[SequencesWithQuality] \
    --input-path results/1-data-prep/manifest.txt \
    --output-path results/1-data-prep/ccs-reads.qza \
    --input-format SingleEndFastqManifestPhred33V2

Imported manifest.txt as SingleEndFastqManifestPhred33V2 to 1-data-prep/ccs-reads.qza


### Visualizing imported reads

In [ ]:
!qiime demux summarize \
    --i-data results/1-data-prep/ccs-reads.qza \
    --o-visualization results/1-data-prep/ccs-reads.qzv

In [ ]:
import qiime2 as q2
q2.Visualization.load("results/1-data-prep/ccs-reads.qzv")

# <font color = 'gray'>Step 2: ASV Inference (Denoising)</font>

[What is DADA2] [What does denoise CCS do?]

In [ ]:
!qiime dada2 denoise-ccs \
    --i-demultiplexed-seqs results/1-data-prep/ccs-reads.qza \
    --p-front AGRGTTYGATYMTGGCTCAG \
    --p-adapter RGYTACCTTGTTACGACTT \
    --p-n-threads 4 \
    --p-min-len 1000 \
    --p-max-len 1600 \
    --o-table results/2-asv-infer/ccs-denoised-table.qza \
    --o-representative-sequences results/2-asv-infer/ccs-denoised-rep-seqs.qza \
    --o-denoising-stats results/2-asv-infer/ccs-denoised-stats.qza \
    --verbose

In [ ]:
!qiime feature-table summarize \
    --i-table results/2-asv-infer/ccs-denoised-table.qza \
    --o-visualization results/2-asv-infer/ccs-denoised-table.qzv

!qiime feature-table tabulate-seqs \
    --i-data results/2-asv-infer/ccs-denoised-rep-seqs.qza \
    --o-visualization results/2-asv-infer/ccs-denoised-rep-seqs.qzv

!qiime metadata tabulate \
    --m-input-file results/2-asv-infer/ccs-denoised-stats.qza \
    --o-visualization results/2-asv-infer/ccs-denoised-stats.qzv

In [ ]:
q2.Visualization.load("results/2-asv-infer/ccs-denoised-table.qzv")

In [ ]:
q2.Visualization.load("results/2-asv-infer/ccs-denoised-rep-seqs.qzv")

In [ ]:
q2.Visualization.load("results/2-asv-infer/ccs-denoised-stats.qzv")

# <font color = 'gray'>Step 3: Taxonomic Assignment</font>

### Annotating against

To annotate the metabarcoding data, we use a reference database which will classify the sequences to their taxonomic identities using the plugin `sci-kit learn`. [Greengenes 16s]

<font color = 'red'>NOTE: Replace the file specified in the <font face = 'Consolas'><b>--i-classifier</b></font> flag by whichever you will use. 

In [ ]:
#Using the green genes classifier to assign taxonomies to the ASV sequences
!qiime feature-classifier classify-sklearn \
    --i-reads results/2-asv-infer/ccs-denoised-rep-seqs.qza \
    --i-classifier assets/2024.09.backbone.full-length.nb.qza \
    --o-classification results/3-tax-assign/asv-taxa.qza 

In [ ]:
#Tabulate predictions
!qiime metadata tabulate \
    --m-input-file results/2-tax-assign/asv-taxa.qza \
    --o-visualization results/2-tax-assign/asv-taxa.qzv

In [ ]:
#Visualize
q2.Visualization.load('results/2-tax-assign/asv-taxa.qzv')

We can view interactive taxonomic barplot to see the composition of each sample.

After loading the visualization, select *Level* to 7 to view at the most resolved classification. You can also toggle the orders of the samples based on their metadata.

In [ ]:
#generate a taxa barplot
!qiime taxa barplot \
    --i-table results/2-asv-infer/ccs-denoised-table.qza \
    --i-taxonomy results/3-tax-assign/asv-taxa.qza \
    --o-visualization results/3-tax-assign/2-bar-plots-asv.qzv

In [ ]:
q2.Visualization.load('results/2-tax-assign/2-bar-plots-asv.qzv')

### Making a phylogenetic tree
[needed for rarefaction]

# <font color = 'gray'>Step 4: Rarefaction</font>

In [ ]:
# Generate a tree for phylogenetic diversity analyses
!qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences results/2-asv-infer/ccs-denoised-rep-seqs.qza \
    --o-alignment results/3-tax-assign/aligned-rep-seqs.qza \
    --o-masked-alignment results/3-tax-assign/masked-aligned-rep-seqs.qza \
    --o-tree results/3-tax-assign/unrooted-tree.qza \
    --o-rooted-tree results/3-tax-assign/rooted-tree.qza

### Making rarefaction curves
[start with 10000] [modify based on graphs]

In [ ]:
!qiime diversity alpha-rarefaction \
    --i-table results/2-asv-infer/ccs-denoised-table.qzv \
    --p-max-depth 10000 \
    --p-metrics 'shannon' \
    --o-visualization results/4-rarefy/asv-table-cleaned_arare.qzv \

# Visualize
q2.Visualization.load('results/4-rarefy/asv-table-cleaned_arare.qzv')

### Rarefying samples
As the rarefaction curves for the samples seem to have plateau-ed in the smallest number of ASVs (i.e., 2329), sampling depth will be set to this number.

In [ ]:
# Alpha and Beta Diversity Analyses
!qiime diversity core-metrics-phylogenetic \
    --i-phylogeny results/3-tax-assign/rooted-tree.qza \
    --i-table results/2-asv-infer/ccs-denoised-table.qzv \
    --p-sampling-depth 2329 \
    --m-metadata-file metadata-day-2.txt \
    --output-dir results/5-diversity/core-metrics-results

# <font color = 'gray'>Step 5: Diversity Analysis</font>

### Visualizing beta diversity indices
After looking for the discontinuity of data using hierarchical clustering, grouping of sites will then be viewed in a multidimensional space.

Run the code blocks below to view the PCoA plot of each beta diversity metric. You can select a <i>color category</i> for any metadata column, for example, the cluster category, to see if any data points group together.

In [ ]:
# Jaccard distance
q2.Visualization.load('results/5-diversity/core-metrics-results/jaccard_emperor.qzv')

In [ ]:
# Bray-Curtis dissimilarity
q2.Visualization.load('results/5-diversity/core-metrics-results/bray_curtis_emperor.qzv')

In [ ]:
# Unweighted Unifrac
q2.Visualization.load('results/5-diversity/core-metrics-results/unweighted_unifrac_emperor.qzv')

In [ ]:
# Weighted Unifrac
q2.Visualization.load('results/5-diversity/core-metrics-results/weighted_unifrac_emperor.qzv')

### Test for statistical significance in alpha diversity 
After visualizing in a multidimensional space and deciding on clusters, significance testing of the groups based on their alpha diversity will be done. This is explored to check if a cluster of sites is significantly more diverse than the other.

Run the code blocks below to test for significance and view the results.

In [ ]:
# Shannon
!qiime diversity alpha-group-significance \
  --i-alpha-diversity results/5-diversity/core-metrics-results/shannon_vector.qza \
  --m-metadata-file metadata-day-2.txt \
  --o-visualization results/5-diversity/2-shannon-group-significance.qzv

q2.Visualization.load('results/5-diversity/2-shannon-group-significance.qzv')

In [ ]:
# Faith's PD
!qiime diversity alpha-group-significance \
  --i-alpha-diversity results/5-diversity/core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file metadata-day-2.txt \
  --o-visualization results/5-diversity/3-faith_pd-group-significance.qzv

q2.Visualization.load('results/5-diversity/3-faith_pd-group-significance.qzv')

### Test for statistical significance in beta diversity
Then, significance testing of the groups based on their beta diversity will be examined. This is done to check if there is a significant difference in the composition of communities between/among groups.

Run the code blocks below to test for significance and view the results.

In [ ]:
# Bray-Curtis
!qiime diversity beta-group-significance \
  --i-distance-matrix results/5-diversity/core-metrics-results/bray_curtis_distance_matrix.qza \
  --m-metadata-file metadata-day-2.txt \
  --m-metadata-column cluster \
  --o-visualization results/5-diversity/4-bray-curtis-cluster-significance.qzv \
  --p-pairwise

q2.Visualization.load('results/5-diversity/4-bray-curtis-cluster-significance.qzv')

In [ ]:
# Unweighted Unifrac
!qiime diversity beta-group-significance \
  --i-distance-matrix results/5-diversity/core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file metadata-day-2.txt \
  --m-metadata-column cluster \
  --o-visualization results/5-diversity/5-unweighted-unifrac-cluster-significance.qzv \
  --p-pairwise

q2.Visualization.load('results/5-diversity/5-unweighted-unifrac-cluster-significance.qzv')